# fastMRI
Python 3.6+, Tensorflow 2.0.0-dev20190601

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, glob
import h5py # for loading .h5 files
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
plt.style.use('seaborn') # pretty matplotlib plots
plt.rcParams["axes.grid"] = False # don't show grid lines on plots by default
plt.rcParams['figure.figsize'] = (12, 16) # increase size of subplots

In [5]:
import cs230_project_utilities as utils

In [6]:
# Sync data:
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_val data/fastmri/raw/singlecoil_dev # (val -> dev) 
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_test data/fastmri/raw/singlecoil_test
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_train data/fastmri/raw/singlecoil_train

In [7]:
# Prepare local directories

raw_dev_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/*.h5'
raw_test_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_test/*.h5'
raw_train_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_train/*.h5'

dev_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_dev'
test_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_test'
train_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_train'

!mkdir -p $dev_tfrecord_directory
!mkdir -p $test_tfrecord_directory
!mkdir -p $train_tfrecord_directory

In [ ]:
# Run conversion

utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_dev_locations,
                                                        dev_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=True,
                                                        perform_subsampling=False)

Converting file 0 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001984.h5' to TFRecords...
Converting file 1 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000323.h5' to TFRecords...
Converting file 2 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000283.h5' to TFRecords...
Converting file 3 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001759.h5' to TFRecords...
Converting file 4 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000976.h5' to TFRecords...
Converting file 5 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000926.h5' to TFRecords...
Converting file 6 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000831.h5' to TFRecords...
Converting file 7 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001429.h5' to TFRecords...
Converting file 8 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000647.h5' to TFRecords...
Converting file 9 at b'/home/ubuntu/cs230/data

Converting file 77 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000033.h5' to TFRecords...
Converting file 78 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000810.h5' to TFRecords...
Converting file 79 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000899.h5' to TFRecords...
Converting file 80 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000942.h5' to TFRecords...
Converting file 81 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001851.h5' to TFRecords...
Converting file 82 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001440.h5' to TFRecords...
Converting file 83 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000328.h5' to TFRecords...
Converting file 84 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000201.h5' to TFRecords...
Converting file 85 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000178.h5' to TFRecords...
Converting file 86 at b'/home/ubuntu/

In [ ]:
utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_test_locations,
                                                        test_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=False,
                                                        perform_subsampling=False)

In [ ]:
utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_train_locations,
                                                        train_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=False,
                                                        perform_subsampling=False)

In [ ]:
# Sync TFRecords to s3
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_dev s3://cs230-automap/data/fastmri/tfrecords/singlecoil_dev
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_test s3://cs230-automap/data/fastmri/tfrecords/singlecoil_test
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_train s3://cs230-automap/data/fastmri/tfrecords/singlecoil_train

# Inspect dev data

In [ ]:
# Load data

data_locations = os.path.join(dev_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction')
utils.plot.imshowgray(y)

# Inspect test data

In [ ]:
# Load data

data_locations = os.path.join(test_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction (fully sampled)')
utils.plot.imshowgray(y)

# Inspect train data

In [ ]:
# Load data

data_locations = os.path.join(train_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction')
utils.plot.imshowgray(y)